In [1]:
import base64
import os
import uuid
import vertexai
from google.adk.agents import LlmAgent
from google.adk.apps import App
from google.adk.artifacts import InMemoryArtifactService
from google.adk.planners import BuiltInPlanner
from google.adk.plugins.save_files_as_artifacts_plugin import SaveFilesAsArtifactsPlugin
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import load_artifacts
from google.genai.types import Part, Content, Blob, ThinkingConfig

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION)

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
root_agent = LlmAgent(
    name='image_analyst_agent',
    model='gemini-2.5-flash',
    instruction='''
Your role is to analyze given image files.
Use load_artifacts() if the image content is not in the context.
''',
    tools=[load_artifacts],
    planner=BuiltInPlanner(
        thinking_config=ThinkingConfig(
            include_thoughts=False,
            thinking_budget=0,
        )
    ),
)

app = App(
    root_agent=root_agent,
    name='iamge_analyzer_app',
    plugins=[SaveFilesAsArtifactsPlugin()],
)

In [3]:
def get_image_data(file_path: str):
    with open(file_path, 'rb') as f:
        image_bytes = f.read()
    return base64.b64encode(image_bytes).decode('utf-8')

client = ChatClient(app)

image_base64 = get_image_data('testimage.png')

parts = [
    Part.from_text(text='Count the number of people in the image.'),
    Part(
        inline_data=Blob(
            mime_type='image/png',
            data=image_base64,
        ),
    ),
]
content = Content(role='user', parts=parts)

_ = await client.stream(content)

NameError: name 'ChatClient' is not defined

In [ ]:
_ = await client.stream('Count the number of dogs in the previous image.')

In [ ]:
session = await client._runner.session_service.get_session(
    app_name = client._app.name,
    user_id = client._user_id,
    session_id = client._session.id
)
for event in session.events:
    print(event)
    print('====')